# Testing EUGENE training

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing the training of EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

# Set-up

In [1]:
import numpy as np
import pandas as pd

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [2]:
from eugene.dataloading.SeqDataModule import SeqDataModule

# Model params

In [3]:
MODEL = "fcn"
STRAND = "ss"
TASK = "regression"

# Instantiate model and data via config

In [10]:
import claim.utils as cu
from eugene.models.fcn import FCN
from eugene.models.cnn import CNN
from eugene.models.rnn import RNN
from eugene.models.hybrid import hybrid

In [11]:
import pytorch_lightning as pl
from pytorch_lightning.utilities.cli import LightningArgumentParser
from pytorch_lightning.loggers import TensorBoardLogger

In [12]:
parser = LightningArgumentParser()
parser.add_lightning_class_args(pl.Trainer, nested_key="trainer")
parser.add_lightning_class_args(SeqDataModule, nested_key="data")
parser.add_lightning_class_args(FCN, nested_key="model");

In [8]:
config = "/cellar/users/aklie/projects/EUGENE/config/full/1_full_fcn.yaml"
yml = parser.parse_path(cfg_path=config)
mod = SeqDataModule(**yml["data"])
eugene = FCN(**yml["model"])

/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [28]:
data_config = "/cellar/users/aklie/projects/EUGENE/config/data/train/2021_OLS_Library_Training_NPY-T_act_reg.yaml"
data_yml = parser.parse_path(cfg_path=data_config)
mod = SeqDataModule(**data_yml["data"])

In [33]:
model_config = "/cellar/users/aklie/projects/EUGENE/config/models/simple/regression/ss_act-reg_hybrid.yaml"

In [34]:
model_yml = parser.parse_path(cfg_path=model_config)
eugene = hybrid(**model_yml["model"])

usage: ipykernel_launcher.py [-h] [--config CONFIG]
                             [--print_config [={comments,skip_null}+]]
                             [--trainer CONFIG] [--trainer.logger LOGGER]
                             [--trainer.checkpoint_callback {true,false,null}]
                             [--trainer.enable_checkpointing {true,false}]
                             [--trainer.callbacks CALLBACKS]
                             [--trainer.default_root_dir DEFAULT_ROOT_DIR]
                             [--trainer.gradient_clip_val GRADIENT_CLIP_VAL]
                             [--trainer.gradient_clip_algorithm GRADIENT_CLIP_ALGORITHM]
                             [--trainer.process_position PROCESS_POSITION]
                             [--trainer.num_nodes NUM_NODES]
                             [--trainer.num_processes NUM_PROCESSES]
                             [--trainer.devices DEVICES] [--trainer.gpus GPUS]
                             [--trainer.auto_select_gpus {true,

Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/jsonargparse/core.py", line 974, in check_required
    raise TypeError
TypeError

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/jsonargparse/core.py", line 1004, in check_config
    check_required(cfg, self)
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch_dev/lib/python3.7/site-packages/jsonargparse/core.py", line 976, in check_required
    raise TypeError(f'Key "{prefix}{reqkey}" is required but not included in config object or its value is None.') from ex
TypeError: Key "data.seq_file" is required but not included in config object or its value is None.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/cellar/users/aklie/opt/miniconda3/envs/pytorch

TypeError: object of type 'NoneType' has no len()

In [29]:
mod.setup()

In [30]:
dataset = mod.train_dataloader().dataset.dataset

In [31]:
dataset.targets

array([0.611767, 0.313011, 0.387121, ..., 0.180546, 0.194393, 0.278342])

In [32]:
mod.load_kwargs

{'rev_seq_file': '/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/ohe_seq/0.09-0.4_X-all_ohe-seq-rev.npy',
 'target_file': '/cellar/users/aklie/projects/EUGENE/data/2021_OLS_Library/activity/0.09-0.4_y-all_activity.txt',
 'is_target_text': True}

# Train the model with a trainer

In [11]:
LOG_DIR = f"../test/test_logs/config"
logger = TensorBoardLogger(LOG_DIR, name=MODEL, version=f"{STRAND}_{TASK}")
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [12]:
trainer.fit(model=eugene, datamodule=mod)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name     | Type                      | Params
-------------------------------------------------------
0 | fcn      | BasicFullyConnectedModule | 42.6 K
1 | accuracy | Accuracy                  | 0     
2 | auroc    | AUROC                     | 0     
-------------------------------------------------------
42.6 K    Trainable params
0         Non-trainable params
42.6 K    Total params
0.170     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Scratch